# Extract all Pan Evaporation

An archive containing high quality rainfall data from the BOM has been downloaded from here:

http://www.bom.gov.au/climate/change/hqsites/about-hq-site-data.shtml 

There is a file with the list of stations and then a zip file for each station containing that station’s data.

Create a notebook that will extract all the folders, and extract all the files, filters just the latest data according the minYearToLoad and Saves the final file into the ./data_files/ folder for future processing


In [6]:
import pandas as pd
import os
from pathlib import Path
from unlzw import unlzw

# For testing, set this a small number to just do a few files. Otherwise, set to 9999 to do unlimited (basically)
maxFiles = 9999
# When extracting the rainfall data per station, set to what year you what to go back to
minYearToLoad = 2014

sourceFolder = "./data_files_raw/Pan_Evaporation/extracted/"
workingSubFolder = "./data_files_raw/Pan_Evaporation/working/"
dataFolder = "./data_files/"
stationsFile = "HQME_stations"

if not os.path.exists(workingSubFolder):
  Path(workingSubFolder).mkdir(parents=True,exist_ok=True)      


if not os.path.exists(dataFolder):
  Path(dataFolder).mkdir(parents=True,exist_ok=True)        

First, load the stations file into a dataframe. Since the delimiters are only spaces, but there are spaces in words, need to do it a bit more manually  

In [7]:
lstStationId = []
lstLatitude = []
lstLongitude = []
lstElevationMetres = []
lstStationName = []

stationName = ""
with open(sourceFolder + stationsFile, "r") as station_file:
  for line in station_file:
    line = line.strip()
    tokens = line.split(" ")

    for i in range(4, len(tokens)):    
      if i == 4:
        stationName = tokens[4]
      else:
        stationName += " " + tokens[i]    

    lstStationId.append(tokens[0])
    lstLatitude.append(float(tokens[1]))
    lstLongitude.append(float(tokens[2]))
    lstElevationMetres.append(float(tokens[3]))    
    lstStationName.append(stationName)

In [8]:
dfStations = pd.DataFrame(
  { 
    "StationId" : lstStationId,
    "Latitude" : lstLatitude,
    "Longitude" : lstLongitude,
    "ElevationMs" : lstElevationMetres, 
    "StationName" : lstStationName
  }  
)

del lstStationId
del lstLatitude
del lstLongitude
del lstElevationMetres
del lstStationName

dfStations.head(10)


,StationId,Latitude,Longitude,ElevationMs,StationName
0,002012,-18.23,127.66,422.0,HALLS CREEK AIRPORT
1,002014,-15.65,128.71,31.0,KIMBERLEY
2,003003,-17.95,122.23,7.0,BROOME AIRPORT
3,004032,-20.37,118.63,6.4,PORT HEDLAND AIRPORT
4,005007,-22.24,114.10,5.0,LEARMONTH AIRPORT
5,005026,-22.24,118.34,463.0,WITTENOOM
6,006011,-24.89,113.67,4.0,CARNARVON AIRPORT
7,007045,-26.61,118.54,517.0,MEEKATHARRA AIRPORT
8,008051,-28.80,114.70,33.0,GERALDTON AIRPORT
9,009021,-31.93,115.98,15.4,PERTH AIRPORT


Loop through each file in extracted. All files that end in .Z, unzip to a subfolder in the working folder

In [9]:
# For testing, set this a small number to just do a few files. Otherwise, set to 9999 to do unlimited (basically)
stepper = 0

lstEvapStationId = []
lstEvapFromYear = []
lstEvapFromMonth = []
lstEvapFromDay = []
lstEvapToYear = []
lstEvapToMonth = []
lstEvapToDay = []
lstEvapValue = []

for filename in os.listdir(sourceFolder):
  if os.path.isfile(sourceFolder+filename) and filename.lower().endswith(".z"):
    if (stepper > maxFiles):
      break
    
    # This is one of the zip files, extract it to a subfolder in the working folder, named after the file
    if not os.path.exists(workingSubFolder + filename):
      Path(workingSubFolder + filename).mkdir(parents=True,exist_ok=True) 

    fh = open(sourceFolder+filename, 'rb')
    compressed_data = fh.read()
    uncompressed_data = unlzw(compressed_data)    
    fh.close()

    fileContents = uncompressed_data.decode("utf-8")
    
    fileLines = fileContents.split("\n")

    if len(fileLines) > 0:
      # The first line is some header, just extract the stationid from it

      firstLine = fileLines[0]
      firstLine = firstLine.replace("EVAP", "").strip()
      stationId = firstLine[0:6]

      for i in range(1, len(fileLines)):        
      #for i in range(1, 3):      
        fileLines[i] = fileLines[i].replace("  ", " ")
        tokens = fileLines[i].split(" ")

        if len(tokens) == 3:

          year = int(tokens[0][0:4])
        
          if year >= minYearToLoad:
            month = int(tokens[0][5:6])
            day = int(tokens[0][7:8])

            yearTo = int(tokens[1][0:4])
            monthTo = int(tokens[1][5:6])
            dayTo = int(tokens[1][7:8])

            value = float(tokens[2])

            lstEvapStationId.append(stationId)
            lstEvapFromYear.append(year)
            lstEvapFromMonth.append(month)
            lstEvapFromDay.append(day)
            lstEvapToYear.append(yearTo)
            lstEvapToMonth.append(monthTo)
            lstEvapToDay.append(dayTo)
            lstEvapValue.append(value)

    stepper += 1



In [10]:
dfEvap = pd.DataFrame(
  { 
    "StationId" : lstEvapStationId,
    "Year" : lstEvapFromYear,
    "Month" : lstEvapFromMonth,
    "Day" : lstEvapFromDay, 
    "YearTo" : lstEvapToYear,
    "MonthTo" : lstEvapToMonth,
    "DayTo" : lstEvapToDay, 
    "Value" : lstEvapValue
  }  
)
dfEvap.head(10)

,StationId,Year,Month,Day,YearTo,MonthTo,DayTo,Value
0,002012,2014,1,1,2014,2,1,2751.6
1,002012,2015,1,1,2015,2,1,2767.1
2,002012,2016,6,1,2016,6,0,99999.9
3,002012,2016,6,1,2016,8,1,99999.9
4,003003,2014,1,1,2014,2,1,2902.1
5,003003,2015,1,1,2015,2,1,2989.0
6,003003,2016,1,1,2016,2,1,3017.1
7,003003,2017,3,1,2017,3,1,99999.9
8,004032,2014,1,1,2014,2,1,3390.4
9,004032,2015,2,1,2015,2,1,99999.9


Join on the Stations

In [11]:
dfMerged = pd.merge(dfStations, dfEvap, on="StationId")

In [12]:
dfMerged.to_csv(dataFolder + "annual_pan_evaporation_" + str(minYearToLoad) + ".csv", index=False)